In [ ]:
class trainmodel:
                    
    def __init__(self):        
        self.train_df = self.test_df = self.Y = pd.DataFrame()
        self.train_df = pd.read_csv("../../data/processed/train.csv")
        self.test_df = pd.read_csv("../../data/processed/test.csv")                    
        
    def transform_target(self):
        
        le  = LabelBinarizer()
        cat_arr = le.fit_transform(self.train_df['cat'])
        cat_df = pd.DataFrame(data=cat_arr,columns=le.classes_)        
        self.train_df['subcat'] = np.where(pd.isnull(self.train_df['subcat']),'NaN',self.train_df['subcat'])
        subcat_arr = le.fit_transform(self.train_df['subcat'])
        subcat_df = pd.DataFrame(data=subcat_arr,columns=le.classes_)    
        self.Y = pd.concat([cat_df,subcat_df],axis=1)        
        self.Y.drop('NaN',inplace=True,axis=1)
        #print('Y value', self.Y.columns)
        
    def prepare_data(self):
        
        col_todrop = ['cat','subcat','Unnamed: 0','document_id']
        for col in col_todrop:
            self.train_df.drop(col,axis=1,inplace=True)          
            
    def prepare_validationset(self):
        
        X_train, X_test, Y_train, Y_test = train_test_split(self.train_df['content'], self.Y, 
                                                            test_size=0.33, random_state=42)                 
        return(X_train, X_test, Y_train, Y_test)

    def createmodel(self):
        
        X_train, X_test, Y_train, Y_test =  self.prepare_validationset()
        #X_train, X_test, Y_train, Y_test = train_test_split(self.train_df['content'], self.Y,test_size=0.33, random_state=42)                             
        X_train_transposed =X_train.T
        self.pipe = Pipeline([
                    ('cv', CountVectorizer(ngram_range=(1, 3))),
                    ('tfidf', TfidfTransformer()),
                    ('LP_GNB', LabelPowerset(GaussianNB()))])
        # train
        self.pipe.fit(X_train_transposed, Y_train)
        
        # predict
        prediction = self.pipe.predict(X_test)
         
        print('validation dataset accuracy is {}'.format(accuracy_score(Y_test, prediction)))           
        print('Hamming loss is {}'.format(hamming_loss(Y_test, prediction)))
        print('F1 Score is {}'.format(f1_score(Y_test, prediction,average='samples')))
        
        return()
    
    def termfreq(self,n_comp):
        
        tfidfvec = TfidfVectorizer(ngram_range=(1,3), strip_accents='unicode',
                       lowercase =True, analyzer='word', token_pattern=r'\w+',
                       stop_words = 'english')
        
        tfidf = tfidfvec.fit_transform(self.train_df['content'])  
        self.train_df.drop('content',inplace=True,axis=1)
        self.train_df = pd.DataFrame(tfidf.toarray(), columns=tfidfvec.get_feature_names())
        
        '''
        df1 = pd.DataFrame(tfidf.toarray(), columns=tfidfvec.get_feature_names())
        self.train_df = pd.concat([self.train_df, df1], axis=1)
        svdT = TruncatedSVD(n_components=n_comp)
        self.train_df = svdT.fit_transform(self.train_df) 
        
        
        test_tfidf = tfidfvec.transform(self.test_df['content'])
        self.test_df.drop('content',inplace=True,axis=1)
        df1 = pd.DataFrame(test_tfidf.toarray(), columns=tfidfvec.get_feature_names())
        self.test_df = pd.concat([self.test_df, df1], axis=1)
        self.test_df = svdT.transform(self.test_tfidf)                 
        '''
        
    def createmodel_2(self):
                        
        self.termfreq()                      
        clf = LabelPowerset(GaussianNB())
        X_train, X_test, Y_train, Y_test =  self.prepare_validationset()
        clf.fit(X_train,Y_train)
        prediction = clf.predict(X_test)
        
        # predict
        # prediction = self.pipe.predict(X_test)
         
        print('validation dataset accuracy is {}'.format(accuracy_score(Y_test, prediction)))           
        print('Hamming loss is {}'.format(hamming_loss(Y_test, prediction)))
        print('F1 Score is {}'.format(f1_score(Y_test, prediction,average='samples')))
        
        return()
    
    def prepare_testdata(self,fname):
        
        self.test_df = pd.read_csv(fname)         
        self.test_id = self.test_df['document_id']
        
        col_todrop = ['document_id','Unnamed: 0']
        for col in col_todrop:
            self.test_df.drop(col,axis=1,inplace=True)            
        
    
    def predict_testdataset(self):
         
        test_predicitions = self.pipe.predict(self.test_df['content'])         
        return(test_predicitions)
    
    
    def get_testdata(self):
        return(self.test_df)
    
    def get_traindata(self):
        return(self.train_df)
    
    def get_target(self):
        return(self.Y)